In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import numpy as np
import sklearn.model_selection as model_selection
TRAIN_LABELS = 'participants/train/labels/labels.csv'
TRAIN_TEXT = 'participants/train/extracted_data/extract_combined.csv'
TEST_TEXT = 'participants/test/extracted_data/extract_combined.csv'


In [2]:
train_labels_df = pd.read_csv(TRAIN_LABELS, usecols=['document_name','is_fitara'])
train_text_df = pd.read_csv(TRAIN_TEXT)
test_df = pd.read_csv(TEST_TEXT)

msk = np.random.rand(train_text_df.shape[0]) < 0.7

In [3]:
train_labels_df.is_fitara = train_labels_df.is_fitara.eq('Yes').mul(1)

train_df = pd.merge(
    train_labels_df, 
    train_text_df, 
    on='document_name', 
    how='inner'
)
train_df.head()

,document_name,is_fitara,text
0,04-42RFP.pdf,0,\n \nOMB No. 0990-0115 \n \n Electronic Requ...
1,1_-_Brand_name_JOFOC_1782798.pdf,0,\n \nJustification for Other than Full and Op...
2,1_-_Brand_Name_JOFOC_for_FBO.docx,0,Justification for Other than Full and Open Com...
3,1_-_Brand_Name_Only.docx,1,Justification for Other than Full and Open Com...
4,1_-_JOFOC_-_FSSI_Limited_Source.doc,1,Attachment C\r\r\rHHS Template and Instruction...


In [40]:
# train_XY = train_df[msk]
# test_XY = train_df[~msk]
train_x, test_x, train_y, test_y = model_selection.train_test_split(train_df.text, train_df.is_fitara, test_size=0.3)

In [44]:
tfidf_vect_ngram = TfidfVectorizer(stop_words='english', analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(train_x)
ngram_train = tfidf_vect_ngram.transform(train_x.values)
ngram_test = tfidf_vect_ngram.transform(test_x.values)

In [45]:
nb_classifier2 = MultinomialNB()
nb_classifier2.fit(ngram_train, train_y)
ngram_pred = nb_classifier2.predict(ngram_test)
ngram_score = metrics.accuracy_score(test_y, ngram_pred)

In [46]:
print('============= NGRAM VECTORIZER ================')
print('Loss = ',metrics.log_loss(test_y, ngram_pred))
print('Score = ',ngram_score)
print('CM = \n',metrics.confusion_matrix(test_y, ngram_pred, labels=[0, 1]))

============= NGRAM VECTORIZER ================
Loss =  7.461410892047552
Score =  0.7839721254355401
CM = 
 [[185  26]
 [ 36  40]]
